In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from etrago import Etrago
from etrago import tools
from etrago.tools.plot import (
    coloring, 
    make_legend_circles_for, 
    make_handler_map_to_scale_circles_as_in,
    plot_grid,
    calc_storage_expansion_per_bus,
)
import matplotlib.patches as mpatches
colors = coloring

Import networks

In [ ]:
etrago_H2share0_lf = Etrago(csv_folder_name= "RES/lf_300_80__8760_5_H2share0/results")
network_H2share0_lf = etrago_H2share0_lf.network.copy()

etrago_ch4_load_abroad = Etrago(csv_folder_name= "RES/results_eGon2035_lowflex_nofeedin_CH4loads")
network_ch4_load_abroad = etrago_ch4_load_abroad.network.copy()

In [ ]:
networks = {
    "low_flex": network_H2share0_lf,
    "CH4_load_abroad": network_ch4_load_abroad,
}

label = {
    "low_flex": "no H2 feedin",
    "CH4_load_abroad": "CH4 load abroad changed",
}

In [ ]:
buses_DE = {}
for key in networks:
    buses_DE[key] = networks[key].buses.loc[networks[key].buses.country == 'DE']

# Tests

In [ ]:
for key in networks:
    print(networks[key])

In [ ]:
for key in networks:
    print(networks[key].buses.carrier.unique())
    # print(networks[key].stores.carrier.unique())
    # print(networks[key].links.carrier.unique())
    # print(networks[key].loads.carrier.unique())
    # print(networks[key].generators.carrier.unique())

    # print(networks[key].buses.carrier.value_counts())
    # print(networks[key].links.carrier.value_counts())
    # print(networks[key].stores.carrier.value_counts())
    # print(networks[key].loads.carrier.value_counts())
    # print(networks[key].generators.carrier.value_counts())

# H2

## H2 production

In [ ]:
SMR = {}
ELY = {}

for key in networks:
    SMR[key] = networks[key].links[(networks[key].links.carrier == "CH4_to_H2")]
    ELY[key] = networks[key].links[(networks[key].links.carrier == "power_to_H2")]

In [ ]:
H2_dispatch_SMR = {}
H2_dispatch_ELY = {}

for key in networks:
    H2_dispatch_SMR[key] = - networks[key].links_t.p1[SMR[key].index]
    H2_dispatch_ELY[key] = - networks[key].links_t.p1[ELY[key].index]

In [ ]:
total_dispatch = {}

fig, ax = plt.subplots(figsize = (10,5))

for key in networks:
    total_dispatch[key] = H2_dispatch_SMR[key].sum(axis = 1) + H2_dispatch_ELY[key].sum(axis = 1)

    (total_dispatch[key]/1e3).resample('20H').mean().plot(
        ax=ax, 
        label=("Total H2 production "+label[key]), 
        legend=True)
    (H2_dispatch_SMR[key]/1e3).sum(axis = 1).resample('20H').mean().plot(
        ax=ax, 
        label=("SMR "+label[key]),
        linestyle='dashed',
        linewidth=0.8,
        legend=True
    )
    (H2_dispatch_ELY[key]/1e3).sum(axis = 1).resample('20H').mean().plot(
        ax=ax, 
        label=("ELY "+label[key]),
        linestyle='dashed',
        linewidth=0.8,
        legend=True
    )


ax.set_ylabel('GW')
ax.set_title('H2 Generation')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

fig.show()

### ELY: power to H2

In [ ]:
for key in networks:
    print(label[key])
    print(f'Total H2 production by power_to_H2 (in GW): {H2_dispatch_ELY[key].sum().sum()/1e3:.2f}')
    print(f'Total power_to_H2 capacity built (in GW): {ELY[key].p_nom_opt.sum()/1e3:.2f}, (with efficiency: {(ELY[key].p_nom_opt.sum()/1e3)*(ELY[key].efficiency.mean()):.2f} GW)')
    print(f'Total SMR capacity built (in GW): {SMR[key].p_nom_opt.sum()/1e3:.2f}, (with efficiency: {(SMR[key].p_nom_opt.sum()/1e3)*(SMR[key].efficiency.mean()):.2f} GW) \n')

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))

for key in networks:
    (H2_dispatch_ELY[key]/1e3).sum(axis = 1).resample('20H').mean().plot(ax=ax, label=(label[key]), legend=True)

ax.set_ylabel('GW')
ax.set_title('ELY dispatch')

fig.show()

#### Spatial repartition

In [ ]:
ely_capacities_h2 = {}
ely_h2_bus = {}

for key in networks:

    ely_h2_bus[key] = ELY[key]["bus1"]
    ely_capacities_h2[key] = H2_dispatch_ELY[key].sum(axis=0)
    new_idx = [ely_h2_bus[key][i] for i in ely_capacities_h2[key].index.to_list()]
    ely_capacities_h2[key] = pd.Series(ely_capacities_h2[key].values,index = new_idx).groupby(level=0).sum()
    
    scaling = 5/1e6

    bus_scaling = 1/1e2
    bus_sizes = ely_capacities_h2[key] * scaling
    bus_unit = "TWh"

    print(bus_sizes.min())
    print(bus_sizes.max())

    handles = make_legend_circles_for([
        # bus_sizes.min()*bus_scaling, 
        bus_sizes.max()*bus_scaling,
        ], 
        scale=1, 
        facecolor=colors()["H2_to_power"]
    )
    labels = [#"min", "max"]
        ("{} " + bus_unit).format(s)
        for s in (
            # round(bus_sizes.min()),
            round(bus_sizes.max()),
        )
    ]

    network = networks[key].copy()
    network.buses = network.buses[
        (network.buses.carrier == "H2_grid")
        # | (network.buses.carrier == "H2_saltcavern")
    ]

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
    network.plot(
        bus_sizes=bus_sizes*bus_scaling,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        bus_colors=colors()["H2_to_power"], 
        title=("Total optimized ELY production at H2 buses in "+label[key]))
    

    l2 = ax.legend(
    handles,
    labels,
    loc='upper left',
    # bbox_to_anchor=(0.01, 1.01),
    labelspacing=1.0,
    framealpha=1.0,
    # title=bus_legend,
    handler_map=make_handler_map_to_scale_circles_as_in(ax),
    )
    ax.add_artist(l2)

    fig.show()

#### Dispatch

In [ ]:
# ely_capacities_sorted_ac = {}
# power_bus_ELY_id  = {}

# for key in networks:
#     ely_capacities_sorted_ac[key] = ely_capacities_ac[key].sort_values(ascending=False)
#     power_bus_ELY_id[key] = [i for i in ely_capacities_sorted_ac[key].index.to_list()[:1]]
#     print(power_bus_ELY_id[key])

In [ ]:
# ELY_t_ac = {}

# for key in networks:
#     # ELY_group = 
#     ELY_t_ac[key] = networks[key].links_t.p0[ELY[key][ELY[key]["bus0"].isin(power_bus_ELY_id[key])].index]
    
#     fig, ax = plt.subplots(figsize = (10,5))
#     for i in ELY_t_ac[key].columns:
#         ELY_t_ac[key][[i]].plot(ax=ax, label=i, legend=True)
#     fig.show()

In [ ]:
# ely_capacities_sorted_h2 = {}
# h2_bus_ELY_id  = {}

# for key in networks:
#     ely_capacities_sorted_h2[key] = ely_capacities_h2[key].sort_values(ascending=False)
#     h2_bus_ELY_id[key] = [i for i in ely_capacities_sorted_h2[key].index.to_list()[:1]]
#     print(h2_bus_ELY_id[key])

In [ ]:
# ELY_t_h2 = {}

# for key in networks:
#     # ELY_group = 
#     ELY_t_h2[key] = - networks[key].links_t.p1[ELY[key][ELY[key]["bus1"].isin(h2_bus_ELY_id[key])].index]
    
#     fig, ax = plt.subplots(figsize = (10,5))
#     for i in ELY_t_h2[key].columns:
#         ELY_t_h2[key][[i]].plot(
#             ax=ax, 
#             label=i,
#             legend=True,
#             )
#     fig.show()

### SMR: CH4 to H2

In [ ]:
for key in networks:
    print(f'Total H2 production by CH4_to_H2 for "{label[key]}" (in GW): {H2_dispatch_SMR[key].sum().sum()/1e3:.3f}')

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))

for key in networks:
    H2_dispatch_SMR[key].sum(axis = 1).resample('20H').mean().plot(ax=ax, label=(label[key]), legend=True)

ax.set_ylabel('MW')
ax.set_title('SMR dispatch')

fig.show()

In [ ]:
capacities = {}
H2_bus = {}
scaling = {}

for key in networks:

    H2_bus[key] = SMR[key]["bus1"]
    capacities[key] = H2_dispatch_SMR[key].sum(axis=0)
    new_idx = [H2_bus[key][i] for i in capacities[key].index.to_list()]
    capacities[key] = pd.Series(capacities[key].values,index = new_idx)
    scaling[key] = 1/capacities[key].max()

    network = networks[key].copy()
    network.buses = network.buses[(network.buses.carrier == "H2_grid")]

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
    network.plot(
        bus_sizes=capacities[key] * scaling[key]/2,#/100000,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        title=("Optimized SMR dispatch for "+label[key]),
    )
    fig.show()


In [ ]:
# capacities_sorted = {}
# H2_bus_SMR_id  = {}

# for key in networks:
#     capacities_sorted[key] = capacities[key].sort_values(ascending=False)
#     H2_bus_SMR_id[key] = [i for i in capacities_sorted[key].index.to_list()[:5]]

# print(H2_bus_SMR_id)

In [ ]:
# SMR_t = {}

# for key in networks:
#     SMR_t[key] = -networks[key].links_t.p1[SMR[key][SMR[key]["bus1"].isin(H2_bus_SMR_id[key])].index]
    
#     fig, ax = plt.subplots(figsize = (10,5))
#     for i in SMR_t[key].columns:
#         SMR_t[key][[i]].plot(ax=ax, label=i, legend=True)
#     fig.show()

## H2 uses

In [ ]:
rel_h2_links_carriers = ['H2_feedin', 'H2_to_CH4', 'H2_to_power']
rel_h2_loads_carriers = ['H2_for_industry', 'H2_hgv_load']

H2_use = {}

for key in networks:
    # Links consuming H2
    H2_use[key] = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == rel_h2_links_carriers[0]].index.to_list()]
    H2_use[key] = pd.DataFrame(H2_use[key].sum(axis=1))
    H2_use[key] = H2_use[key].rename(columns={0:rel_h2_links_carriers[0]})

    for i in rel_h2_links_carriers[1:]:
        h2_loads = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == i].index.to_list()]
        H2_use[key][i] = h2_loads.sum(axis=1)

    # H2 loads
    DE_loads = networks[key].loads.loc[networks[key].loads.bus.isin(buses_DE[key].index)]
    for i in rel_h2_loads_carriers:
        h2_loads = networks[key].loads_t.p[DE_loads.loc[DE_loads.carrier == i].index.to_list()]
        H2_use[key][i] = h2_loads.sum(axis=1)
    
    # Plot
    fig = plt.figure(figsize=(10,5)) #, dpi=300
    (H2_use[key].resample('20H').mean()/1e3).plot.area(
        ax = plt.gca(), 
        title='Stacked H2 Loads by carrier '+label[key], 
        ylabel = "[GW]", 
        legend=True, 
        # stacked=True,
    )
    fig.show()

# H2 to CH4

## H2 feedin

In [ ]:
h2_feedin_links = {}
feedin_dispatch = {}
ch4_links = {}
ch4_dispatch = {}

for key in networks:
    h2_feedin_links[key] = networks[key].links[(networks[key].links.carrier == "H2_feedin")]
    feedin_dispatch[key] = networks[key].links_t.p0[h2_feedin_links[key].index]
    ch4_links[key] = networks[key].links[(networks[key].links.carrier == "CH4")]
    ch4_dispatch[key] = (
        abs(networks[key].links_t.p0[ch4_links[key].index])
        + abs(networks[key].links_t.p1[ch4_links[key].index])
    )/2
    
    # print(label[key])
    # print(f'Total feedin capacity (static): {h2_feedin_links[key].p_nom.sum()/1e3:.2f} GW')
    # print(f'Maximal use (%): {feedin_dispatch[key].sum(axis = 1).max()/h2_feedin_links[key].p_nom.sum()*100:.3}')
    # print(f'Total use H2 feedin (TWh): {feedin_dispatch[key].sum(axis = 1).sum()/1e6:.2f}\n')

### Spatial repartition

In [ ]:
feedin_capacities = {}
feedin_H2_bus = {}
feedin_scaling = {}

for key in networks:

    feedin_H2_bus[key] = h2_feedin_links[key]["bus0"]
    feedin_capacities[key] = feedin_dispatch[key].sum(axis=0)
    new_idx = [feedin_H2_bus[key][i] for i in feedin_capacities[key].index.to_list()]
    feedin_capacities[key] = pd.Series(feedin_capacities[key].values,index = new_idx)
    feedin_scaling[key] = 1/feedin_capacities[key].max()

    network = networks[key].copy()
    network.buses = network.buses[(network.buses.carrier == "H2_grid")]

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
    network.plot(
        bus_sizes=feedin_capacities[key] * feedin_scaling[key]/3,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        title=("H2 feedin dispatch for "+label[key]),
    )
    fig.show()

### Share of methane flow

H2_vol2en = {
    0: 0,
    2: 0.00612,
    5: 0.01562,
    10: 0.03242,
    15: 0.05053,
    20: 0.07011,
}

In [ ]:
rel_ch4_loads_carriers = ['rural_gas_boiler', 'CH4_for_industry', 'CH4']
rel_h2_loads_carriers = ['H2_for_industry', 'H2_hgv_load']

In [ ]:
# for key in ["ch4_load_lim"]: #    " #  "cst_0_1": network_cst_0_1,
#     for i in h2_feedin_links[key].index: #[:1]: # h2_feedin_links[key].index: # ['4696']: #feedin_links[key].index: #
#         h2_bus_id = h2_feedin_links[key].at[i,"bus0"]
#         ch4_bus_id = h2_feedin_links[key].at[i,"bus1"]
#         feedin_at_ch4_bus = - networks[key].links_t.p1[i]

#         ch4_links_0 = networks[key].links[
#             (networks[key].links.carrier == "CH4")
#             & (networks[key].links.bus0 == ch4_bus_id)
#         ]
#         ch4_links_1 = networks[key].links[
#             (networks[key].links.carrier == "CH4")
#             & (networks[key].links.bus1 == ch4_bus_id)
#         ]

#         ch4_exchange = abs(
#             (networks[key].links_t.p0[ch4_links_0.index]).sum(axis = 1) 
#             - (networks[key].links_t.p1[ch4_links_1.index]).sum(axis = 1)
#         )
#         # ratio = feedin_at_ch4_bus/(feedin_at_ch4_bus+ch4_exchange)*100

#         # CH4 generation
#         ch4_gen_gen = networks[key].generators[
#             (networks[key].generators.bus == ch4_bus_id)
#             & ((networks[key].generators.carrier == "CH4_NG") | (networks[key].generators.carrier == "CH4_biogas"))
#         ]
#         ch4_gen_meth = networks[key].links[
#             (networks[key].links.carrier == "H2_to_CH4")
#             & (networks[key].links.bus1 == ch4_bus_id)
#         ]
#         ch4_gen = networks[key].generators_t.p[ch4_gen_gen.index].sum(axis = 1) - networks[key].links_t.p1[ch4_gen_meth.index].sum(axis = 1)
        
#         # ELY at H2 bus
#         ely_at_h2_bus = ELY[key][ELY[key]["bus1"] == h2_bus_id]
#         ely_at_h2_bus_dispatch = - networks[key].links_t.p1[ely_at_h2_bus.index].sum(axis = 1)

#         # H2 and CH4 loads
#         H2_loads = networks[key].loads[(networks[key].loads.bus == h2_bus_id) & (networks[key].loads.carrier.isin(rel_h2_loads_carriers))]
#         CH4_loads = networks[key].loads[(networks[key].loads.bus == ch4_bus_id) & (networks[key].loads.carrier.isin(rel_ch4_loads_carriers))]
#         h2_load_dispatch = networks[key].loads_t.p[H2_loads.index].sum(axis=1)
#         ch4_load_dispatch = networks[key].loads_t.p[CH4_loads.index].sum(axis=1)
        
#         # Static capacities (non optimized)
#         feedin_cap = h2_feedin_links[key].at[i,"p_nom"]
#         ch4_cap = ch4_links_1.p_nom.sum() + ch4_links_0.p_nom.sum()
#         ratio_cap = feedin_cap/ch4_cap

#         ch4_flow_total = ch4_exchange + ch4_load_dispatch + ch4_gen

#         # Plot
#         fig, ax1 = plt.subplots() #figsize=(15,5)
#         # fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20,5))
        
#         (feedin_at_ch4_bus/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "H2 feedin",
#             # color='green',
#             # legend=True,
#         )
#         (ch4_flow_total/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "Total CH4 flow (exchange through pipelines + loads + generation)",
#             # color='green',
#             # legend=True,
#         )
#         (ch4_exchange/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "CH4 flow in gas pipelines", 
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         (ch4_load_dispatch/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "CH4 load",
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         (ch4_gen/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = f"CH4 generation at CH4 bus ({ch4_bus_id}) (methanisation + extraction)",
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         ax1.set_ylabel('GW')
#         ax1.set_title(
#             f"""
#                 Flow at CH4 bus {ch4_bus_id} {label[key]}
#                 Total p_nom CH4 =  {(ch4_cap/1e3):.2f} GW, p_nom H2 feedin = {(feedin_cap/1e3):.2f} GW, ratio p_nom = {ratio_cap:.3f}""", 
#                 fontsize=9,)
#         ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
#         fig.show()

In [ ]:
# for key in ["cst_0_1"]: #    " #  "cst_0_1": network_cst_0_1,
#     for i in h2_feedin_links[key].index[:3]: #[:1]: # h2_feedin_links[key].index: # ['4696']: #feedin_links[key].index: #
#         h2_bus_id = h2_feedin_links[key].at[i,"bus0"]
#         ch4_bus_id = h2_feedin_links[key].at[i,"bus1"]
#         feedin_at_ch4_bus = - networks[key].links_t.p1[i]

#         ch4_links_0 = networks[key].links[
#             (networks[key].links.carrier == "CH4")
#             & (networks[key].links.bus0 == ch4_bus_id)
#         ]
#         ch4_links_1 = networks[key].links[
#             (networks[key].links.carrier == "CH4")
#             & (networks[key].links.bus1 == ch4_bus_id)
#         ]

#         ch4_exchange = abs(
#             (networks[key].links_t.p0[ch4_links_0.index]).sum(axis = 1) 
#             - (networks[key].links_t.p1[ch4_links_1.index]).sum(axis = 1)
#         )
#         # ratio = feedin_at_ch4_bus/(feedin_at_ch4_bus+ch4_exchange)*100

#         # CH4 generation
#         ch4_gen_gen = networks[key].generators[
#             (networks[key].generators.bus == ch4_bus_id)
#             & ((networks[key].generators.carrier == "CH4_NG") | (networks[key].generators.carrier == "CH4_biogas"))
#         ]
#         ch4_gen_meth = networks[key].links[
#             (networks[key].links.carrier == "H2_to_CH4")
#             & (networks[key].links.bus1 == ch4_bus_id)
#         ]
#         ch4_gen = networks[key].generators_t.p[ch4_gen_gen.index].sum(axis = 1) - networks[key].links_t.p1[ch4_gen_meth.index].sum(axis = 1)
        
#         # ELY at H2 bus
#         ely_at_h2_bus = ELY[key][ELY[key]["bus1"] == h2_bus_id]
#         ely_at_h2_bus_dispatch = - networks[key].links_t.p1[ely_at_h2_bus.index].sum(axis = 1)

#         # H2 and CH4 loads
#         H2_loads = networks[key].loads[(networks[key].loads.bus == h2_bus_id) & (networks[key].loads.carrier.isin(rel_h2_loads_carriers))]
#         CH4_loads = networks[key].loads[(networks[key].loads.bus == ch4_bus_id) & (networks[key].loads.carrier.isin(rel_ch4_loads_carriers))]
#         h2_load_dispatch = networks[key].loads_t.p[H2_loads.index].sum(axis=1)
#         ch4_load_dispatch = networks[key].loads_t.p[CH4_loads.index].sum(axis=1)
        
#         # Static capacities (non optimized)
#         feedin_cap = h2_feedin_links[key].at[i,"p_nom"]
#         ch4_cap = ch4_links_1.p_nom.sum() + ch4_links_0.p_nom.sum()
#         ratio_cap = feedin_cap/ch4_cap

#         ch4_flow_total = ch4_exchange + ch4_load_dispatch + ch4_gen

#         # Plot
#         fig, ax1 = plt.subplots() #figsize=(15,5)
#         # fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20,5))
        
#         (feedin_at_ch4_bus/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "H2 feedin",
#             # color='green',
#             # legend=True,
#         )
#         (ch4_flow_total/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "Total CH4 flow (exchange through pipelines + loads + generation)",
#             # color='green',
#             # legend=True,
#         )
#         (ch4_exchange/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "CH4 flow in gas pipelines", 
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         (ch4_load_dispatch/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = "CH4 load",
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         (ch4_gen/1e3).resample('20H').mean().plot(
#             ax=ax1, 
#             label = f"CH4 generation at CH4 bus ({ch4_bus_id}) (methanisation + extraction)",
#             linestyle='dashed',
#             linewidth=0.8,
#             # legend=True,
#         )
#         ax1.set_ylabel('GW')
#         ax1.set_title(
#             f"""
#                 Flow at CH4 bus {ch4_bus_id} {label[key]}
#                 Total p_nom CH4 =  {(ch4_cap/1e3):.2f} GW, p_nom H2 feedin = {(feedin_cap/1e3):.2f} GW, ratio p_nom = {ratio_cap:.3f}""", 
#                 fontsize=9,)
#         ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
#         fig.show()

# CH4

## CH4 production

In [ ]:
h2_feedin_links = {}
methanisation = {}
ch4_gen_abroad = {}
ch4_gen_DE = {}
ch4_gen = {}
ch4_gen_biogas = {}
ch4_gen_NG = {}

ch4_carriers = ['CH4', 'CH4_NG', 'CH4_biogas']
ch4_carriers_DE = ['CH4_NG', 'CH4_biogas']

for key in networks:
    # h2_feedin_links[key] = networks[key].links[(networks[key].links.carrier == "H2_feedin")]
    methanisation[key] = networks[key].links[(networks[key].links.carrier == "H2_to_CH4")]
    ch4_gen_abroad[key] = networks[key].generators[(networks[key].generators.carrier == "CH4")]
    ch4_gen_DE[key] = networks[key].generators[(networks[key].generators.carrier.isin(ch4_carriers_DE))]
    ch4_gen[key] = networks[key].generators[(networks[key].generators.carrier.isin(ch4_carriers))]
    ch4_gen_biogas[key] = networks[key].generators[(networks[key].generators.carrier == "CH4_biogas")]
    ch4_gen_NG[key] = networks[key].generators[(networks[key].generators.carrier == "CH4_NG")]

In [ ]:
CH4_dispatch_methanisation = {}
# CH4_dispatch_H2_feedin = {}
CH4_gen_dispatch = {}
CH4_gen_dispatch_DE = {}
CH4_gen_dispatch_abroad = {}
CH4_store_dispatch = {}

for key in networks:
    CH4_dispatch_methanisation[key] = - networks[key].links_t.p1[methanisation[key].index]
    # CH4_dispatch_H2_feedin[key] = - networks[key].links_t.p1[h2_feedin_links[key].index]
    CH4_gen_dispatch[key] = networks[key].generators_t.p[[col for col in networks[key].generators_t.p.columns if 'CH4' in col]]
    CH4_gen_dispatch_DE[key] = networks[key].generators_t.p[ch4_gen_DE[key].index]
    CH4_gen_dispatch_abroad[key] = networks[key].generators_t.p[ch4_gen_abroad[key].index]
    CH4_store_dispatch[key] = networks[key].stores_t.p[[col for col in networks[key].stores_t.p.columns if 'CH4' in col]]

In [ ]:
total_dispatch_ch4_prod = {}

for key in networks:
    fig, ax = plt.subplots(figsize = (10,5))
    total_dispatch_ch4_prod[key] = CH4_dispatch_methanisation[key].sum(axis = 1) + CH4_gen_dispatch[key].sum(axis = 1)
    # total_dispatch_ch4_prod[key] = CH4_dispatch_methanisation[key].sum(axis = 1) + CH4_dispatch_H2_feedin[key].sum(axis = 1) + CH4_gen_dispatch[key].sum(axis = 1)

    (CH4_gen_dispatch_DE[key]/1e3).sum(axis = 1).resample('20H').mean().plot(
        ax=ax, 
        label=("CH4 generation in Germany"), 
        legend=True
    )
    (CH4_gen_dispatch_abroad[key]/1e3).sum(axis = 1).resample('20H').mean().plot(
        ax=ax, 
        label=("CH4 generation abroad"), 
        legend=True
    )
    (CH4_dispatch_methanisation[key]/1e3).sum(axis = 1).resample('20H').mean().plot(
        ax=ax, label=("methanisation"), 
        legend=True
    )
    (total_dispatch_ch4_prod[key]/1e3).resample('20H').mean().plot(
        ax=ax, 
        label=("Total CH4 production"), 
        legend=True
    )
    # (CH4_dispatch_H2_feedin[key]/1e3).sum(axis = 1).resample('20H').mean().plot(ax=ax, label=("H2 feedin"), legend=True)
    # (CH4_gen_dispatch[key]/1e3).sum(axis = 1).resample('20H').mean().plot(ax=ax, label=("CH4 generation"), legend=True)
    # (CH4_store_dispatch[key]/1e3).sum(axis = 1).resample('20H').mean().plot(ax=ax, label=("CH4 store dispatch"), legend=True)

    ax.set_ylabel('GW')
    ax.set_title('CH4 Production '+label[key])

    fig.show()

### Spatial repartition

#### Abroad

In [ ]:
for key in networks:
    ch4_gen_capacities = networks[key].generators_t.p[ch4_gen[key].index].sum(axis=0)
    new_idx = [i.replace(' CH4_biogas','').replace(' CH4_NG','').replace(' CH4','') for i in ch4_gen_capacities.index.to_list()]
    ch4_gen_capacities = pd.Series(ch4_gen_capacities.values,index = new_idx).groupby(level=0).sum()

    ch4_meth_bus = methanisation[key]["bus1"]
    meth_capacities = CH4_dispatch_methanisation[key].sum(axis=0)
    meth_capacities.index =(ch4_meth_bus.values)
    # new_idx = [ch4_meth_bus[i] for i in meth_capacities.index.to_list()]
    # meth_capacities = pd.Series(ely_capacities_ac[key].values,index = new_idx).groupby(level=0).sum()
    
    ## Only abroad
    # ch4_gen_capacities = networks[key].generators_t.p[ch4_gen_abroad[key].index].sum(axis=0)
    # new_idx = [i.replace(' CH4','') for i in ch4_gen_capacities.index.to_list()]  # remove 'CH4' from index to be able to use network.plot
    # ch4_gen_capacities = pd.Series(ch4_gen_capacities.values,index = new_idx)

    # scaling = 1/ch4_gen_capacities.max()

    scaling = 5/1e6
    bus_scaling = 1/1e2
    bus_sizes_gen = ch4_gen_capacities * scaling
    bus_sizes_meth = meth_capacities * scaling
    bus_unit = "TWh"
 
    handles = make_legend_circles_for(
        # [bus_sizes.min()*bus_scaling, bus_sizes.max()*bus_scaling],
        [bus_sizes_gen.max()*bus_scaling, bus_sizes_meth.max()*bus_scaling], 
        scale=1, 
        facecolor="grey",# colors()["H2_to_CH4"]] #-> yellow :/
    )
    labels = [f"{round(bus_sizes.max())} " + bus_unit]
    # labels = [#"min", "max"]
    #     ("{} " + bus_unit).format(s)
    #     for s in (
    #         round(bus_sizes.min()),
    #         round(bus_sizes.max()),
    #     )
    # ]

    # network = networks[key].copy()
    # network.buses = network.buses[
    #     (network.buses.country != "RU")
    #     & (network.buses.carrier == "CH4")
    # ]
    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, figsize = (15,7))
    networks[key].plot(
        bus_sizes=bus_sizes_gen * bus_scaling,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        bus_colors="grey",#colors()["CH4"],
        # title = "Optimized CH4 generation "+label[key]
    )
    networks[key].plot(
        bus_sizes=bus_sizes_meth * bus_scaling,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        bus_colors=colors()["H2_to_CH4"],
        title = "Optimized CH4 generation "+label[key],
    )
    
    l2 = ax.legend(
        handles,
        labels,
        loc='upper left',
        # bbox_to_anchor=(0.01, 1.01),
        labelspacing=1.5,
        framealpha=1.0,
        title="Total generated CH4",
        handler_map=make_handler_map_to_scale_circles_as_in(ax),
    )
    ax.add_artist(l2)
    
    handles = []
    color={"CH4 generators": "grey", "methanisation":colors()["H2_to_CH4"]}
    
    for i in ["CH4 generators", "methanisation"]:
        patch = mpatches.Patch(color=color[i], label=i)
        handles.append(patch)

    l3 = plt.legend(handles=handles, loc="center left", ncol=3,)# bbox_to_anchor=(-0.1, 0)
    ax.add_artist(l3)
    
    fig.show()

#### In Germany

In [ ]:
for key in networks:
    ch4_gen_capacities = networks[key].generators_t.p[ch4_gen_DE[key].index].sum(axis=0)

    new_idx = [i.replace(' CH4_biogas','').replace(' CH4_NG','') for i in ch4_gen_capacities.index.to_list()]  #.replace(' CH4_NG','')# remove 'CH4' from index to be able to use network.plot
    ch4_gen_capacities = pd.Series(ch4_gen_capacities.values,index = new_idx).groupby(level=0).sum()
    scaling = 1/ch4_gen_capacities.max()

    network = networks[key].copy()
    network.buses = network.buses[
        (network.buses.country == "DE")
        & (network.buses.carrier == "CH4")
    ]

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}) #, figsize = (15,7)
    network.plot(
        bus_sizes=ch4_gen_capacities * scaling,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        title = "Optimized CH4 generation in Germany "+label[key])
    
    fig.show()

### Constraint

In [ ]:
print("Rq: Results are multiplied by 5 because of skip_snapshots\n")
for key in networks:
    ch4_gen_NG = networks[key].generators[networks[key].generators.carrier == "CH4_NG"]
    ch4_gen_NG_d = networks[key].generators_t.p[ch4_gen_NG.index].sum(axis=0)

    ch4_gen_biogas = networks[key].generators[networks[key].generators.carrier == "CH4_biogas"]
    ch4_gen_biogas_d = networks[key].generators_t.p[ch4_gen_biogas.index].sum(axis=0)

    print(f'Total NG produced in Germany ({label[key]}): {5*ch4_gen_NG_d.sum()/1e6:.2f} TWh')
    print(f'Total biogas produced in Germany ({label[key]}): {5*ch4_gen_biogas_d.sum()/1e6:.2f} TWh')

## CH4 uses

In [ ]:
rel_ch4_links_carriers = network.links.loc[network.links.bus0.isin(network.buses.loc[network.buses.carrier == 'CH4'].index)].carrier.unique()
rel_ch4_links_carriers = np.delete(rel_ch4_links_carriers, np.where(rel_ch4_links_carriers == 'CH4'))
rel_ch4_loads_carriers = ['rural_gas_boiler', 'CH4_for_industry', 'CH4']

CH4_use = {}
total_dispatch_ch4_store = {}

for key in networks:
    # Links consuming CH4
    CH4_use[key] = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == rel_ch4_links_carriers[0]].index.to_list()]
    CH4_use[key] = pd.DataFrame(CH4_use[key].sum(axis=1))
    CH4_use[key] = CH4_use[key].rename(columns={0:rel_ch4_links_carriers[0]})

    for i in rel_ch4_links_carriers[1:]:
        ch4_loads = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == i].index.to_list()]
        CH4_use[key][i] = ch4_loads.sum(axis=1)

    # CH4 loads
    DE_loads = networks[key].loads.loc[networks[key].loads.bus.isin(buses_DE[key].index)]
    for i in rel_ch4_loads_carriers:
        ch4_loads = networks[key].loads_t.p[DE_loads.loc[DE_loads.carrier == i].index.to_list()]
        CH4_use[key][i] = ch4_loads.sum(axis=1)

    # Stores
    total_dispatch_ch4_store[key] = CH4_store_dispatch[key].sum(axis=1)
    
    # Plot
    fig = plt.figure(figsize=(10,5)) #, dpi=300
    (CH4_use[key].resample('20H').mean()/1e3).plot.area( #.area
        ax = plt.gca(), 
        title='Stacked CH4 Loads by carrier '+label[key], 
        ylabel = "[GW]", 
        legend=True, 
        stacked=True,
    )
    # ((total_dispatch_ch4_prod[key] + total_dispatch_ch4_store[key])/1e3).resample('20H').mean().plot(
    #     ax = plt.gca(), 
    #     legend = True, 
    #     label='Total generation and storage dispatch (CH4)', 
    #     color='black', 
    #     linestyle='dashed')
    
    fig.show()

In [ ]:
rel_ch4_links_carriers = network.links.loc[network.links.bus0.isin(network.buses.loc[network.buses.carrier == 'CH4'].index)].carrier.unique()
rel_ch4_links_carriers = np.delete(rel_ch4_links_carriers, np.where(rel_ch4_links_carriers == 'CH4'))
rel_ch4_loads_carriers = ['rural_gas_boiler', 'CH4_for_industry', 'CH4']

CH4_use = {}
total_dispatch_ch4_store = {}

for key in networks:
    # Links consuming CH4
    CH4_use[key] = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == rel_ch4_links_carriers[0]].index.to_list()]
    CH4_use[key] = pd.DataFrame(CH4_use[key].sum(axis=1))
    CH4_use[key] = CH4_use[key].rename(columns={0:rel_ch4_links_carriers[0]})

    for i in rel_ch4_links_carriers[1:]:
        ch4_loads = networks[key].links_t.p0[networks[key].links.loc[networks[key].links.carrier == i].index.to_list()]
        CH4_use[key][i] = ch4_loads.sum(axis=1)

    # CH4 loads
    DE_loads = networks[key].loads.loc[networks[key].loads.bus.isin(buses_DE[key].index)]
    for i in rel_ch4_loads_carriers:
        ch4_loads = networks[key].loads_t.p[DE_loads.loc[DE_loads.carrier == i].index.to_list()]
        CH4_use[key][i] = ch4_loads.sum(axis=1)

    # Stores
    total_dispatch_ch4_store[key] = CH4_store_dispatch[key].sum(axis=1)
    
    # Plot
    fig = plt.figure(figsize=(10,5)) #, dpi=300
    (CH4_use[key].resample('20H').mean()/1e3).plot( #.area
        ax = plt.gca(), 
        title='Stacked CH4 Loads by carrier '+label[key], 
        ylabel = "[GW]", 
        legend=True, 
        # stacked=True,
    )
    # ((total_dispatch_ch4_prod[key] + total_dispatch_ch4_store[key])/1e3).resample('20H').mean().plot(
    #     ax = plt.gca(), 
    #     legend = True, 
    #     label='Total generation and storage dispatch (CH4)', 
    #     color='black', 
    #     linestyle='dashed')
    
    fig.show()

# Stores

In [ ]:
gas_stores_carriers = ['CH4', 'H2_overground', 'H2_underground']

for key in networks:
    print(label[key])
    for i in gas_stores_carriers:
        gas_stores_DE = networks[key].stores.loc[
            (networks[key].stores.carrier == i)
            & (networks[key].stores.bus.isin(buses_DE[key].index))
        ]

        print(f'{i} total storage capacity in Germany:{gas_stores_DE.e_nom_opt.sum()/1e6:.3} TWh')
        print(f'{i} Maximum fill in Germany (%): {networks[key].stores_t.e[gas_stores_DE.index].sum(axis = 1).max()/gas_stores_DE.e_nom_opt.sum()*100:.3}')
        
        (networks[key].stores_t.e[gas_stores_DE.index].sum(axis = 1)/1e6).plot(
            title = 'Stored energy capacity',
            figsize=(10,5), 
            ylabel='[TWh]',
            label = (i+" "+label[key]),
            legend= True,
        )
    fig.show()

## H2 storages

In [ ]:
h2_stores_carriers = ['H2_overground', 'H2_underground']

In [ ]:
for key in networks:
    print(key)
    for i in h2_stores_carriers:
        h2_stores = networks[key].stores.loc[
            (networks[key].stores.carrier == i)
        ]

        print(f'{i} total storage capacity:{h2_stores.e_nom_opt.sum():.2f} MWh')
        print(f'{i} Maximum fill (%): {networks[key].stores_t.e[h2_stores.index].sum(axis = 1).max()/h2_stores.e_nom_opt.sum()*100:.2f} \n')
        
        # (networks[key].stores_t.e[stores.index].sum(axis = 1)/1e6).plot(title = 'Current Storage energy capacity',figsize=(20,10), ylabel='[TWh]',label = i, legend= True)


In [ ]:
h2_stores = {}
h2_stores_dispatch = {}

fig, ax = plt.subplots(figsize = (10,5))

for key in networks: #["5H2share"]: # 

    h2_stores[key] = networks[key].stores.loc[networks[key].stores.carrier.isin(h2_stores_carriers)]
    h2_stores_dispatch[key] = networks[key].stores_t.p[h2_stores[key].index].sum(axis=1)
    
    (h2_stores_dispatch[key]).resample('20H').mean().plot(ax=ax, label=(label[key]), legend=True)

    ax.set_ylabel('MW')
    ax.set_title('H2 stores dispatch')

    fig.show()

### Spatial repartition

In [ ]:
for key in networks:
    h2_store_cap = h2_stores[key].set_index("bus")["e_nom_opt"]

    network = networks[key].copy()
    network.buses = network.buses[(network.buses.carrier.isin(["H2_grid", "H2_saltcavern"]))]
    scaling = 1/h2_store_cap.max()

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
    network.plot(
        bus_sizes=h2_store_cap * scaling/2,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        title="H2 stores for "+label[key],
    )
    fig.show()

## CH4 stores

In [ ]:
CH4_stores_DE = {}
CH4_stores_dispatch_DE = {}

for key in networks:
    CH4_stores_DE[key] = networks[key].stores.loc[(networks[key].stores.carrier=='CH4') & (networks[key].stores.bus.isin(buses_DE[key].index))]
    CH4_stores_dispatch_DE[key] = networks[key].stores_t.e[CH4_stores_DE[key].index].sum(axis=1) 

    (CH4_stores_dispatch_DE[key]/1e6).resample('20H').mean().plot(
        title = 'CH4 global store state in Germany',
        figsize=(10,5), 
        ylabel='[TWh]',
        label = label[key],
        legend= True,
    )

In [ ]:
for key in networks:

    fig, ax = plt.subplots(figsize = (10,5))

    (CH4_stores_dispatch_DE[key]/1e3).resample('20H').mean().plot(
        title = 'CH4 store dispatch',
        ylabel='GW',
    )

    fig.show()

### Spatial repartition

In [ ]:
for key in networks:
    # ch4_store_cap = CH4_stores_DE[key][(CH4_stores[key].bus.isin(buses_DE[key].index))]
    ch4_store_cap = CH4_stores_DE[key].set_index("bus")["e_nom"]

    network = networks[key].copy()
    network.buses = network.buses[
        (network.buses.carrier == "CH4")
        & (network.buses.index.isin(buses_DE[key].index))
    ]
    scaling = 1/ch4_store_cap.max()

    fig, ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()})
    network.plot(
        bus_sizes=ch4_store_cap * scaling/2,
        line_widths=0,
        link_widths=0,
        margin=0.1,
        ax=ax,
        title="CH4 stores for "+label[key],
    )
    fig.show()

## Other stores

In [ ]:
stores_exp = {}
for key in networks:
    stores_exp[key] = calc_storage_expansion_per_bus(networks[key])

In [ ]:
# for key in networks:
#     store_ext = networks[key].stores[networks[key].stores.e_nom_extendable == True].groupby(['carrier']).sum().e_nom_opt
#     store_ext["battery"] = networks[key].storage_units[networks[key].storage_units.carrier == "battery"].p_nom_opt.sum()
#     store_ext = store_ext.sort_values()

#     fig, ax = plt.subplots()
#     # fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,) # figsize=(20,5)
#     (store_ext/1e3).plot.bar(ax)
#     ax.set_ylabel('MWh')
#     ax.set_title('Total optimized storages capacity')
#     ax.legend()
#     plt.show()

#     store_ext.pop("central_heat_store")

#     fig, ax = plt.subplots()

#     (store_ext/1e3).plot.bar(ax)
#     ax.set_ylabel('MWh')
#     ax.set_title('Total optimized storages capacity (without central heat stores)')
#     ax.legend()
#     plt.show()
